In [2]:
import pandas as pd
import seaborn as sb
from sklearn.preprocessing import LabelEncoder

### 1. Understanding dataset


In [4]:
df = pd.read_csv("../../data/lending_club_loan_data.csv")

print(df.head())

print(f"Dataset shape: {df.shape}")

                 emp_title  emp_length state homeownership  annual_income  \
0  global config engineer          3.0    NJ      MORTGAGE        90000.0   
1   warehouse office clerk        10.0    HI          RENT        40000.0   
2                 assembly         3.0    WI          RENT        40000.0   
3         customer service         1.0    PA          RENT        30000.0   
4     security supervisor         10.0    CA          RENT        35000.0   

   verified_income  debt_to_income  annual_income_joint  \
0         Verified           18.01                  NaN   
1     Not Verified            5.04                  NaN   
2  Source Verified           21.15                  NaN   
3     Not Verified           10.16                  NaN   
4         Verified           57.96              57000.0   

  verification_income_joint  debt_to_income_joint  ...  sub_grade  \
0                       NaN                   NaN  ...         C3   
1                       NaN                 

#### 1.2 Check for missing values & fill fix them

- think about possible reasons why data is missing


In [6]:
print(df.isnull().sum())

emp_title                            833
emp_length                           817
state                                  0
homeownership                          0
annual_income                          0
verified_income                        0
debt_to_income                        24
annual_income_joint                 8505
verification_income_joint           8545
debt_to_income_joint                8505
delinq_2y                              0
months_since_last_delinq            5658
earliest_credit_line                   0
inquiries_last_12m                     0
total_credit_lines                     0
open_credit_lines                      0
total_credit_limit                     0
total_credit_utilized                  0
num_collections_last_12m               0
num_historical_failed_to_pay           0
months_since_90d_late               7715
current_accounts_delinq                0
total_collection_amount_ever           0
current_installment_accounts           0
accounts_opened_

## Possible reason for missing values

#### 1.emp_title (833 missing values)

    - borrowers choose not to disclose job title due to privacy concerns

    impact:
        - affect analysis related to employment sector and correlation to credit risk

#### 2. emp_length (817 missing values)

    - borrowers may omit this info if they have short employment history, self-employed or in informal / unverified jobs

    impact:
        - affect features correlating employment stability with default risk

#### 3. annual_income_joint, verification_income_joint, debt_to_income_joint (8505 - 8545 missing values)

    - specific to joint loan applicants
    - if borrower applied for individual loans, these fields are irrelevant

    impact:
        - missing values are expected and not representative of incomplete data

#### 4. months_since_last_delinq (5658 missing values)

    - possibly due to borrowers not having any delinquency history
    - inconsistent reporting / unverified borrower credit records can also lead to missing

    impact:
        - no delinquencies are lower risk but absence of this feature might limit credit behaviour analysis

#### 5. months_since_90d_late (7715 missing values)

    - missing values suggest that borrowers may not have record of being 90d late in credit payment
    - lack of reporting in credit history may also cause this issue

    impact:
        - missing values likely means borrower no severe delinquency but such assumption might lead to underestimation of risks for unreported cases

#### 6. debt_to_income (24 missing values)

    - borrowers did not provide complete information during loan application process
    - borrowers with atypical financial situation

    impact:
        - missing values could bias models & impact reliability


# Resolving Missing Values


In [15]:
df['verification_income_joint'].unique()

array([nan, 'Verified', 'Not Verified', 'Source Verified'], dtype=object)

5.930306000217794

In [ ]:
print(df['emp_title'].unique())
print()
print(df['emp_length'].mean())
print(df['emp_length'].median())
print()
print()


['global config engineer ' 'warehouse office clerk' 'assembly' ...
 'inspector/packer' 'da coordinator ' 'toolmaker']

5.930306000217794
6.0


In [20]:
# categorical, use unknown
df['emp_title'] = df['emp_title'].fillna("unknown")

# mean is 5.93, median is 6. both is fine
df['emp_length'] = df['emp_length'].fillna(df['emp_length'].median())

# 0 to represent not applicable
df['annual_income_joint'] = df['annual_income_joint'].fillna(0)
df['verification_income_joint'] = df['verification_income_joint'].fillna("Not Applicable")
df['debt_to_income_joint'] = df['debt_to_income_joint'].fillna(0)

# assume no delinquency history
df['months_since_last_delinq'] = df['months_since_last_delinq'].fillna(0)


# assume 0 for borrowers without severe delinquency
df['months_since_90d_late'] = df['months_since_90d_late'].fillna(0)  # or 999

# dataset sufficiently large, can drop missing rows
df = df.dropna(subset=['debt_to_income'])


In [22]:
print(df.isnull().sum())
print(df.shape)

emp_title                              0
emp_length                             0
state                                  0
homeownership                          0
annual_income                          0
verified_income                        0
debt_to_income                         0
annual_income_joint                    0
verification_income_joint              0
debt_to_income_joint                   0
delinq_2y                              0
months_since_last_delinq               0
earliest_credit_line                   0
inquiries_last_12m                     0
total_credit_lines                     0
open_credit_lines                      0
total_credit_limit                     0
total_credit_utilized                  0
num_collections_last_12m               0
num_historical_failed_to_pay           0
months_since_90d_late                  0
current_accounts_delinq                0
total_collection_amount_ever           0
current_installment_accounts           0
accounts_opened_

### 3 Summary Statistics & Data description

- identify features that are categorical, numerical, data/time
- generate summary statistics for numerical features to identify potential anomalies


In [23]:
print(df.dtypes)

numerical_features = df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = df.select_dtypes(include='object').columns

print(f"Numerical Features: {numerical_features}")
print(f"Categorical Features: {categorical_features}")

emp_title                            object
emp_length                          float64
state                                object
homeownership                        object
annual_income                       float64
verified_income                      object
debt_to_income                      float64
annual_income_joint                 float64
verification_income_joint            object
debt_to_income_joint                float64
delinq_2y                             int64
months_since_last_delinq            float64
earliest_credit_line                  int64
inquiries_last_12m                    int64
total_credit_lines                    int64
open_credit_lines                     int64
total_credit_limit                    int64
total_credit_utilized                 int64
num_collections_last_12m              int64
num_historical_failed_to_pay          int64
months_since_90d_late               float64
current_accounts_delinq               int64
total_collection_amount_ever    

In [24]:
print(df.describe())
print()
print(df[categorical_features].describe())

        emp_length  annual_income  debt_to_income  annual_income_joint  \
count  9976.000000   9.976000e+03     9976.000000         9.976000e+03   
mean      5.936147   7.941274e+04       19.308192         1.888665e+04   
std       3.553393   6.469524e+04       15.004851         5.277172e+04   
min       0.000000   3.000000e+03        0.000000         0.000000e+00   
25%       3.000000   4.500000e+04       11.057500         0.000000e+00   
50%       6.000000   6.500000e+04       17.570000         0.000000e+00   
75%      10.000000   9.500000e+04       25.002500         0.000000e+00   
max      10.000000   2.300000e+06      469.090000         1.100000e+06   

       debt_to_income_joint    delinq_2y  months_since_last_delinq  \
count           9976.000000  9976.000000               9976.000000   
mean               2.951748     0.216018                 15.964014   
std                7.740674     0.684045                 23.141431   
min                0.000000     0.000000             

In [27]:
df['loan_status'].unique()

array(['Current', 'Fully Paid', 'In Grace Period', 'Late (31-120 days)',
       'Charged Off', 'Late (16-30 days)'], dtype=object)

In [28]:
df.columns

Index(['emp_title', 'emp_length', 'state', 'homeownership', 'annual_income',
       'verified_income', 'debt_to_income', 'annual_income_joint',
       'verification_income_joint', 'debt_to_income_joint', 'delinq_2y',
       'months_since_last_delinq', 'earliest_credit_line',
       'inquiries_last_12m', 'total_credit_lines', 'open_credit_lines',
       'total_credit_limit', 'total_credit_utilized',
       'num_collections_last_12m', 'num_historical_failed_to_pay',
       'months_since_90d_late', 'current_accounts_delinq',
       'total_collection_amount_ever', 'current_installment_accounts',
       'accounts_opened_24m', 'months_since_last_credit_inquiry',
       'num_satisfactory_accounts', 'num_accounts_120d_past_due',
       'num_accounts_30d_past_due', 'num_active_debit_accounts',
       'total_debit_limit', 'num_total_cc_accounts', 'num_open_cc_accounts',
       'num_cc_carrying_balance', 'num_mort_accounts',
       'account_never_delinq_percent', 'tax_liens', 'public_record_bankr